# Set up dask client


In [28]:
! ls -l ../dataset/

total 212595512
-rw-r--r--  1 sarmadtarar  staff      3258527 23 Apr 16:47 dataset_sample.csv
-rw-r--r--@ 1 sarmadtarar  staff    176473873 27 Apr 13:30 groupby_results_weekdayy_groupby_means.csv
-rw-r--r--@ 1 sarmadtarar  staff    169368141 27 Apr 13:30 groupby_results_weekend_groupby_means.csv
-rw-r--r--@ 1 sarmadtarar  staff     10023125 13 May 00:01 qut_it_capstone_ailbatross_20181029_nmi_attributes_20181029.csv
-rw-r--r--@ 1 sarmadtarar  staff  39929480142 24 Apr 17:12 qut_it_capstone_ailbatross_20181029_nmi_data_20181029.csv
-rw-r--r--@ 1 sarmadtarar  staff  28630797979 25 Apr 16:20 weekday_split_weekdays.csv
-rw-r--r--@ 1 sarmadtarar  staff  11298682512 26 May 15:00 weekday_split_weekends.csv
-rw-r--r--  1 sarmadtarar  staff  28630798272 26 May 15:35 weekdays.csv
-rw-r--r--  1 sarmadtarar  staff          293 26 May 15:20 weekends.csv


In [29]:
import dask.dataframe as dd
from dask.distributed import Client, progress

client = Client()

df = dd.read_csv('../dataset/weekdays.csv')

df.head()

/Users/sarmadtarar/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


,nmi_uuid,date,ekw1,ekw2,ekw3,ekw4,ekw5,ekw6,ekw7,ekw8,...,ekw39,ekw40,ekw41,ekw42,ekw43,ekw44,ekw45,ekw46,ekw47,ekw48
0,ed8a8330-7dae-4ebe-a5c8-6f9404b9ed97,2018-02-02,12.878,11.258,11.846,10.282,9.586,10.650,8.616,9.348,...,9.980,8.648,17.250,15.578,14.862,12.120,13.018,12.630,11.714,11.038
1,ed8a8330-7dae-4ebe-a5c8-6f9404b9ed97,2018-02-05,11.068,10.196,8.880,10.640,9.146,8.776,9.810,9.266,...,11.784,15.524,22.438,14.316,12.956,11.568,10.316,10.942,11.650,7.732
2,ed8a8330-7dae-4ebe-a5c8-6f9404b9ed97,2018-02-06,8.408,8.276,8.082,7.562,7.676,8.670,9.786,7.710,...,18.232,21.290,16.606,13.560,13.220,10.822,7.986,8.724,14.650,12.438
3,ed8a8330-7dae-4ebe-a5c8-6f9404b9ed97,2018-02-07,11.356,8.378,7.642,7.968,8.014,10.208,7.710,7.770,...,17.352,12.572,23.524,14.068,11.142,10.330,11.682,8.610,10.744,11.776
4,ed8a8330-7dae-4ebe-a5c8-6f9404b9ed97,2018-02-08,14.958,11.414,10.576,8.614,9.104,8.466,7.346,8.824,...,18.964,15.404,18.312,13.852,16.940,13.834,13.374,8.116,9.794,8.380


In [30]:
nmi_column = df.columns[0]
data_columns = df.columns[2:]
means = df.groupby('nmi_uuid')[[f'ekw{i+1}' for i in range(48)]].mean()

means = client.compute(means)
progress(means)

VBox()

In [7]:
means.result().to_csv('../dataset/weekend_data.csv')

print('Done!')

Done!


# Read in reduced dataset to memory and perform clustering

In [13]:
import pandas as pd

df = pd.read_csv('dataset/weekend_groupby_means.csv')

df.head()

,nmi_uuid,ekw1,ekw2,ekw3,ekw4,ekw5,ekw6,ekw7,ekw8,ekw9,...,ekw39,ekw40,ekw41,ekw42,ekw43,ekw44,ekw45,ekw46,ekw47,ekw48
0,01501e2a-6bbe-4bfd-918e-2d85eee45f8e,2.357118,2.284066,2.341474,2.279237,2.326474,2.249197,2.249421,2.271526,2.274974,...,3.849382,3.660750,3.476434,3.244750,2.937066,2.868092,2.711763,2.638605,2.518171,2.458934
1,01a31d80-6a98-429d-93c6-567ff11077e2,0.316128,0.314039,0.312552,0.310631,0.310128,0.307507,0.302207,0.303094,0.323675,...,1.119281,0.908995,0.867409,0.738197,0.731419,0.577882,0.391133,0.332778,0.320995,0.309833
2,02a13550-4dd8-45ee-9a10-94e185600587,0.557483,0.542103,0.525299,0.542736,0.522287,0.518770,0.512379,0.513115,0.513011,...,1.570023,1.502414,1.508793,1.718839,2.052046,2.169425,1.950230,1.451322,1.000989,0.722414
3,02cc9895-6187-4811-9360-a1ad1d0f7ac4,0.357214,0.323571,0.319643,0.245500,0.317714,0.324571,0.251714,0.296500,0.264929,...,1.540929,1.209429,1.324857,1.227857,1.102643,0.804857,0.694571,0.549429,0.511857,0.539714
4,02fd804c-7a3a-4013-b6dd-dfeef7a8ced4,0.446537,0.480722,0.457019,0.425074,0.425648,0.404056,0.380056,0.311222,0.356889,...,0.621148,0.589259,0.691426,0.866463,0.769426,0.641759,0.585056,0.574852,0.584259,0.523593
